In [1]:
from gym import spaces
from gym.envs.classic_control.cartpole import CartPoleEnv
import numpy as np
import math
from tensorflow.python import keras
from tensorflow.keras.optimizers import Adam

In [2]:
# オリジナルのCartpoleを継承
class MyCartpole(CartPoleEnv):
    def __init__(self):
        super().__init__()

        # action_space を連続空間に変更
        self.action_space = spaces.Box(-self.force_mag, self.force_mag, shape=(1,), dtype=np.float32)

    def reset(self):
        # 終了ターンのが実装内で定義されているので実装
        self.step_count = 0
        return super().reset()

    def step(self, action):
        # 200ターンたったら終了する
        self.step_count += 1
        if self.step_count > 200:
            return np.array(self.state), 0.0, True, {}
        
        # 例外処理
        if np.isnan(action):
            return np.array(self.state), 0.0, True, {}
        
        # アクションをclipして force の値にする
        force = np.clip(action, -self.force_mag, self.force_mag)[0]

        #--- 以下オリジナルのstepコードをコピペ ---

        x, x_dot, theta, theta_dot = self.state
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (force + self.polemass_length * theta_dot ** 2 * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (self.length * (4.0 / 3.0 - self.masspole * costheta ** 2 / self.total_mass))
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == 'euler':
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        done = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned done = True. You "
                    "should always call 'reset()' once you receive 'done = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "Taxi-v3",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 2,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    },
}

In [13]:
env = MyCartpole()
# env = gnwrapper.LoopAnimation(env)

env.reset()     # ゲームの初期化
done = False
total_reward = 0
step = 0

while not done:
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)
    total_reward += reward
    step += 1
env.close()

print("step: {}, reward: {}".format(step, total_reward))

step: 24, reward: 24.0


In [3]:
# 独自のモデルを定義
class PolicyModel(keras.Model):
    def __init__(self, action_space):
        super().__init__()

        # 各レイヤーを定義
        self.dense1 = keras.layers.Dense(16, activation="relu")
        self.dense2 = keras.layers.Dense(16, activation="relu")
        self.pi_mean = keras.layers.Dense(action_space, activation="linear")
        self.pi_stddev = keras.layers.Dense(action_space, activation="linear")

        # optimizer もついでに定義しておく
        self.optimizer = Adam(lr=0.01)

    # Forward pass
    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dense2(x)
        mean = self.pi_mean(x)
        stddev = self.pi_stddev(x)

        # σ^2 > 0 になるように変換(指数関数)
        stddev = tf.exp(stddev)

        return mean, stddev

    # 状態を元にactionを算出
    def sample_action(self, state):
        # モデルから平均と分散を取得
        mean, stddev = self(state.reshape((1,-1)))

        # ガウス分布に従った乱数をだす
        sampled_action = tf.random.normal(tf.shape(mean), mean=mean, stddev=stddev)
        return sampled_action.numpy()[0]